# Zadanie II: Strażnik Cyberbezpieczeństwa – Ochrona Systemu

W tym zadaniu analizuję logi serwera w celu wykrycia potencjalnych ataków hakerskich. Celem jest stworzenie modelu detekcji anomalii, który pozwoli na identyfikację nietypowych zachowań w systemie. W pierwszej kolejności przeprowadzę eksplorację danych, aby zbadać potencjalne cechy i wyczuć co będzie wazne podczas projektowania i trenowania modeli statystycznych jak i sieci neuronowych.

## Plan działania

1. Eksploracja i wizualizacja danych
2. Rozpoznanie podejrzanych wzorców i potencjalnych anomalii
3. Opracowanie modelu statystycznego
4. Rozdzielenie zestawu danych na treningowy, walidacyjny i testowy, wydzielając anomalie do zbioru testowego
5. Feature engineering, preprocessing, etc...
6. Opracowanie sieci neuronowej
7. Trenowanie oraz walidacja sieci neuronowej
8. Test skuteczności na testowym zbiorze danych.

# 1. Eksploracja danych

Zacznę od przejrzenia z jakimi danymi mam doczynienia

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('seaborn-v0_8-dark')
sns.set_palette("husl")

# Read the CSV file
df = pd.read_csv('data/logs.csv', header=None, 
                 names=['date', 'time', 'ip', 'method', 'path', 'protocol', 
                        'status', 'referrer', 'user_agent', 'payload']
                 )

# Combine date and time into a datetime column
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'], 
                               format='%d/%b/%Y %H:%M:%S')

# Basic info about the dataset
print("Dataset Info:")
print(df.info())
print("\nSample of the data:")
print(df.head())
print("\nTail of the data:")
print(df.tail(1))

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14945 entries, 0 to 14944
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        14945 non-null  object        
 1   time        14945 non-null  object        
 2   ip          14945 non-null  object        
 3   method      14945 non-null  object        
 4   path        14945 non-null  object        
 5   protocol    14945 non-null  object        
 6   status      14945 non-null  int64         
 7   referrer    11677 non-null  object        
 8   user_agent  14945 non-null  object        
 9   payload     5128 non-null   object        
 10  timestamp   14945 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 1.3+ MB
None

Sample of the data:
          date      time             ip method        path  protocol  status  \
0  06/Jan/2025  06:02:58  82.81.156.133    GET           /  HTTP/1.1     200  

### Charakterystyka danych

Mamy doczynienia z zrzutem 14945 logów sieciowych serwera `kingbank.pl` na przełomie 10 dni. Jest to niewielka liczba danych. Jak na logi aplikacji są całkiem nieźle ustruktyryzowane i zostały juz prawdopodbnie początkowo wyabstraktowane. 

1. Ruch sieciowy składa się jedynie z ruchu **HTTP**
2. Są to logi poszczególnych punktów końcowych API na serwerze hostującym stronę kingbank.pl
3. Liczba wierszy: **14945**
4. Okres czasowy: **06.01.2025 - 16.01.2025**

In [30]:
# count unique values for each column
print("\nUnique values for each column:")
print(df.nunique())


Unique values for each column:
date             11
time          10002
ip             2194
method            2
path           1004
protocol          1
status            5
referrer        958
user_agent       30
payload        3014
timestamp     10869
dtype: int64


#### Dzięki liście unikalnych wartości widzimy: 

- większość wartości dla payload jest unikalna (3014/5128). 
- Protokół jest tylko jeden: HTTP, nie istotny jako cecha dla modeli. 
- Wyjątkowo mało jest unikalnych user_agent. (mozliwe ze bottom user-agent pokaze coś ciekawego)
- Unikalnych IP jest ~1/7 (2194/14945) zbioru danych. Sugeruje, ze sesje nie trwały długo
- Ciekawe jest, ze unikalnych znaczników czasowych jest tylko 2/3 zbioru danych, mozliwe ataki siłowe

## Przydatne wizualizacje danych

In [ ]:
# Create a figure with multiple subplots
plt.figure(figsize=(20, 12))

# 1. Request Methods Distribution
plt.subplot(2, 2, 1)
method_counts = df['method'].value_counts()
sns.barplot(x=method_counts.index, y=method_counts.values)
plt.title('Distribution of HTTP Methods')
plt.xticks(rotation=45)

# 2. Status Codes Distribution
plt.subplot(2, 2, 2)
status_counts = df['status'].value_counts()
sns.barplot(x=status_counts.index, y=status_counts.values)
plt.title('Distribution of HTTP Status Codes')

# 3. Requests Over Time
plt.subplot(2, 2, 3)
df.set_index('timestamp').resample('1H')['method'].count().plot()
plt.title('Number of Requests per Hour')
plt.xlabel('Time')
plt.ylabel('Number of Requests')
plt.xticks(rotation=45)

# 4. Top 10 Most Requested Paths
plt.subplot(2, 2, 4)
path_counts = df['path'].value_counts().head(10)
sns.barplot(x=path_counts.values, y=path_counts.index)
plt.title('Top 10 Most Requested Paths')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nTop 10 IP addresses by number of requests:")
print(df['ip'].value_counts().head(10))

print("\nTop 10 paths with their request counts:")
print(df['path'].value_counts().head(10))

### Wnioski na temat powyzszych wizualizacji i statystyk

Jak widać nie ma zadnego adresu IP, który miałby nienaturalnie duzą liczbę zapytań. 

Natomiast na pewno punkt końcowy /logowanie ma nienaturalnie duzą liczbę uderzeń, większą niz strona główna aplikacji, jest to ewidentnie nienaturalne zachowanie, a nawet potencjalny atak. Zobaczmy dokładniej jak wyglądają wzorce zapytań i czy mozna znaleźć podejrzane zachowania.  

In [31]:
# Function to detect potential suspicious patterns based, author: claude.ai, editor: 70ziko
def analyze_suspicious_patterns(df):
    suspicious_patterns = {
        'rapid_requests': [],
        'login_attempts': [],
        'suspicious_payloads': [],
        'unusual_paths': []
    }
    
    # 1. Detect rapid requests from same IP (more than 10 requests per minute)
    ip_time_groups = df.groupby([df['ip'], 
                                df['timestamp'].dt.floor('min')])['method'].count()
    rapid_requests = ip_time_groups[ip_time_groups > 7]
    
    # 2. Analyze login attempts - specifically looking at failed login patterns
    login_attempts = df[df['path'].str.contains('/logowanie', na=False)]
    failed_logins = login_attempts[
        login_attempts['status'] // 100 >= 4]
    
    # 3. Look for suspicious payloads - using safe pattern matching
    suspicious_patterns = [
        r'script\W*', r'eval\s*\(', r'exec\s*\(', r'SELECT\s+\w+', 
        r'UNION\s+SELECT', r'DROP\s+TABLE', r'alert\s*\(', 
        r'<[^>]*script'
    ]
    pattern = '|'.join(suspicious_patterns)
    suspicious_payloads = df[
        df['payload'].str.contains(pattern, na=False, case=False, regex=True)]
    
    # 4. Detect unusual paths
    unusual_path_patterns = [
        r'\.\.//', r'%00', r'/etc/passwd', r'/wp-admin',
        r'\.php$', r'\.asp$', r'\.exe$', r'\.dll$'
    ]
    pattern = '|'.join(unusual_path_patterns)
    unusual_paths = df[
        (df['path'].str.len() > 100) | 
        (df['path'].str.contains(pattern, na=False, case=False, regex=True))]
    
    # Additional analysis - Looking for sequential failed login attempts
    sequential_login_attempts = []
    for ip in df['ip'].unique():
        ip_logins = login_attempts[login_attempts['ip'] == ip]
        if len(ip_logins) > 5:  # More than 5 login attempts from same IP
            sequential_login_attempts.append({
                'ip': ip,
                'count': len(ip_logins),
                'timespan': ip_logins['timestamp'].max() - ip_logins['timestamp'].min()
            })
    
    return {
        'rapid_requests': rapid_requests,
        'login_attempts': failed_logins,
        'suspicious_payloads': suspicious_payloads,
        'unusual_paths': unusual_paths,
        'sequential_logins': sequential_login_attempts
    }

# Analyze suspicious patterns
suspicious_patterns = analyze_suspicious_patterns(df)

# Create visualization for the analysis
# plt.figure(figsize=(20, 12))

# # 1. Plot login attempts over time
# plt.subplot(2, 2, 1)
# login_attempts = df[df['path'] == '/logowanie']
# login_attempts.set_index('timestamp')['ip'].resample('1H').count().plot()
# plt.title('Login Attempts Over Time')
# plt.xlabel('Time')
# plt.ylabel('Number of Attempts')

# # 2. Plot unique IPs making login attempts
# plt.subplot(2, 2, 2)
# top_login_ips = login_attempts['ip'].value_counts().head(10)
# sns.barplot(x=top_login_ips.values, y=top_login_ips.index)
# plt.title('Top 10 IPs by Login Attempts')
# plt.xlabel('Number of Attempts')


# plt.tight_layout()
# plt.show()

# Print summary statistics and findings
print("\nSecurity Analysis Summary:")
print(f"Total number of login attempts: {len(login_attempts)}")
print(f"Number of unique IPs making login attempts: {login_attempts['ip'].nunique()}")
print(f"Number of rapid request incidents: {len(suspicious_patterns['rapid_requests'])}")
print(f"Number of suspicious payloads detected: {len(suspicious_patterns['suspicious_payloads'])}")
print(f"Number of unusual paths accessed: {len(suspicious_patterns['unusual_paths'])}")

# Print details of sequential login attempts
print("\nSequential Login Attempts Analysis:")
for attempt in suspicious_patterns['sequential_logins']:
    print(f"IP: {attempt['ip']}")
    print(f"Total login attempts: {attempt['count']}")
    print(f"Timespan: {attempt['timespan']}")
    print("-" * 50)

# Additional analysis of paths accessed by suspicious IPs
suspicious_ips = set(suspicious_patterns['rapid_requests'].index)
if suspicious_ips:
    print("\nPath patterns for suspicious IPs:")
    suspicious_ip_paths = df[df['ip'].isin(suspicious_ips)]['path'].value_counts()
    print(suspicious_ip_paths.head(10))


Security Analysis Summary:
Total number of login attempts: 4087
Number of unique IPs making login attempts: 2023
Number of rapid request incidents: 3
Number of suspicious payloads detected: 3
Number of unusual paths accessed: 0

Sequential Login Attempts Analysis:
IP: 178.144.125.23
Total login attempts: 10
Timespan: 0 days 00:00:57
--------------------------------------------------
IP: 83.9.37.128
Total login attempts: 18
Timespan: 0 days 00:01:03
--------------------------------------------------

Path patterns for suspicious IPs:
Series([], Name: count, dtype: int64)


In [32]:
# Analiza User-Agent
print("\nTop 10 User-Agents:")
print(df['user_agent'].value_counts().head(10))

# Analiza podejrzanych payloadów
print("\nPrzykłady podejrzanych payloadów:")
if len(suspicious_patterns['suspicious_payloads']) > 0:
    print(suspicious_patterns['suspicious_payloads']['payload'].head())
else:
    print("Nie znaleziono podejrzanych payloadów")

# Analiza statusów HTTP dla endpointu /logowanie
login_status = df[df['path'] == '/logowanie']['status'].value_counts()
print("\nRozkład kodów statusu dla /logowanie:")
print(login_status)


Top 10 User-Agents:
user_agent
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36           4435
Mozilla/5.0 (Windows NT 11.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0                                          1615
Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0                                          1574
Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0                                          1510
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36                      430
Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36       418
Mozilla/5.0 (Windows NT 10.0; WOW64; rv:128.0) Gecko/20100101 Firefox/128.0                                                414
Mozilla/5.0 (Linux; Android 13; SM-G990B) AppleWebKit/537.36 (KHTML, like Gecko

## Szczegółowa analiza anomalii i ataków

Na podstawie przeprowadzonej analizy zidentyfikowano następujące typy ataków i anomalii:

In [33]:
def analyze_sql_injection_attempts(df):
    # SQL injection patterns
    sql_patterns = [
        r'UNION\s+SELECT',
        r'SELECT\s+.*\s+FROM',
        r'DROP\s+TABLE',
        r'--\s*$',
        r';\s*--',
        r'OR\s+1=1',
        r'SLEEP\s*\(',
        r'WAITFOR\s+DELAY',
        r'BENCHMARK\s*\('
    ]
    
    pattern = '|'.join(sql_patterns)
    sql_injection_attempts = df[df['payload'].str.contains(pattern, na=False, case=False, regex=True)]
    
    if len(sql_injection_attempts) > 0:
        # Group by IP and get time ranges
        sql_attacks = []
        for ip in sql_injection_attempts['ip'].unique():
            ip_attempts = sql_injection_attempts[sql_injection_attempts['ip'] == ip]
            sql_attacks.append({
                'ip': ip,
                'count': len(ip_attempts),
                'start_time': ip_attempts['timestamp'].min(),
                'end_time': ip_attempts['timestamp'].max(),
                'example_payloads': ip_attempts['payload'].head(3).tolist()
            })
        
        return sql_attacks
    
    return []

def analyze_brute_force_attempts(df):
    login_attempts = df[df['path'].str.contains('/logowanie', na=False)]
    failed_logins = login_attempts[login_attempts['status'] // 100 >= 4]
    
    brute_force_attacks = []
    for ip in failed_logins['ip'].unique():
        ip_attempts = failed_logins[failed_logins['ip'] == ip]
        if len(ip_attempts) >= 5:  # Consider it brute force if 5 or more failed attempts
            brute_force_attacks.append({
                'ip': ip,
                'count': len(ip_attempts),
                'start_time': ip_attempts['timestamp'].min(),
                'end_time': ip_attempts['timestamp'].max(),
                'affected_users': ip_attempts['payload'].str.extract(r'login["\']?\s*[:=]\s*["\']?([^"\',\s]+)', expand=False).dropna().unique().tolist()
            })
    
    return brute_force_attacks

# Analyze SQL injection attempts
sql_attacks = analyze_sql_injection_attempts(df)
print("\nSQL Injection Attacks:")
print("-" * 80)
for attack in sql_attacks:
    print(f"Attacker IP: {attack['ip']}")
    print(f"Number of attempts: {attack['count']}")
    print(f"Time range: {attack['start_time']} to {attack['end_time']}")
    print("Example payloads:")
    for payload in attack['example_payloads']:
        print(f"  - {payload}")
    print("-" * 80)

# Analyze brute force attempts
brute_force_attacks = analyze_brute_force_attempts(df)
print("\nBrute Force Attacks:")
print("-" * 80)
for attack in brute_force_attacks:
    print(f"Attacker IP: {attack['ip']}")
    print(f"Number of attempts: {attack['count']}")
    print(f"Time range: {attack['start_time']} to {attack['end_time']}")
    if attack['affected_users']:
        print(f"Targeted users: {', '.join(attack['affected_users'])}")
    print("-" * 80)


SQL Injection Attacks:
--------------------------------------------------------------------------------
Attacker IP: 178.144.125.23
Number of attempts: 6
Time range: 2025-01-09 09:23:04 to 2025-01-09 09:23:58
Example payloads:
  - login=marpow4372' OR 'x'='x' LIMIT 1;--&password=&action=login
  - login=test'; DROP TABLE users;--&password=&action=login
  - login=marpow4372' UNION SELECT username, password FROM accounts WHERE '1'='1&password=&action=login
--------------------------------------------------------------------------------

Brute Force Attacks:
--------------------------------------------------------------------------------
Attacker IP: 178.144.125.23
Number of attempts: 10
Time range: 2025-01-09 09:23:01 to 2025-01-09 09:23:58
Targeted users: admin, marpow4372, test, marpow4372&password=&action=login, marpow43721
--------------------------------------------------------------------------------


In [34]:
def identify_anomalous_data(df, sql_attacks, brute_force_attacks):
    # Collect all suspicious IPs
    suspicious_ips = set(
        [attack['ip'] for attack in sql_attacks] +
        [attack['ip'] for attack in brute_force_attacks]
    )
    
    # Get time ranges for anomalous activity
    anomaly_periods = []
    for attack in sql_attacks + brute_force_attacks:
        anomaly_periods.append({
            'start': attack['start_time'],
            'end': attack['end_time']
        })
    
    # Mark records as anomalous if they're from suspicious IPs or within anomaly periods
    df['is_anomalous'] = False
    
    # Mark by suspicious IPs
    df.loc[df['ip'].isin(suspicious_ips), 'is_anomalous'] = True
    
    # Mark by time periods
    for period in anomaly_periods:
        mask = (df['timestamp'] >= period['start']) & (df['timestamp'] <= period['end'])
        df.loc[mask, 'is_anomalous'] = True
    
    # Calculate percentage of anomalous data
    anomaly_percentage = (df['is_anomalous'].sum() / len(df)) * 100
    
    print(f"\nAnomaly Statistics:")
    print(f"Total records: {len(df)}")
    print(f"Anomalous records: {df['is_anomalous'].sum()}")
    print(f"Percentage of anomalous data: {anomaly_percentage:.2f}%")
    
    # If less than 20% is anomalous, include additional suspicious behavior
    if anomaly_percentage < 20:
        # Include rapid requests
        ip_time_groups = df.groupby([df['ip'], df['timestamp'].dt.floor('min')])['method'].count()
        rapid_request_ips = ip_time_groups[ip_time_groups > 7].index.get_level_values(0)
        df.loc[df['ip'].isin(rapid_request_ips), 'is_anomalous'] = True
        
        # Include suspicious user agents
        suspicious_agents = df['user_agent'].value_counts()[df['user_agent'].value_counts() < 5].index
        df.loc[df['user_agent'].isin(suspicious_agents), 'is_anomalous'] = True
        
        # Recalculate percentage
        anomaly_percentage = (df['is_anomalous'].sum() / len(df)) * 100
        print(f"\nAfter including additional suspicious behavior:")
        print(f"Anomalous records: {df['is_anomalous'].sum()}")
        print(f"Percentage of anomalous data: {anomaly_percentage:.2f}%")
    
    return df

# Identify anomalous data
df_with_anomalies = identify_anomalous_data(df, sql_attacks, brute_force_attacks)

# Show distribution of anomalies over time
# plt.figure(figsize=(15, 6))
# df_with_anomalies.set_index('timestamp').resample('1H')['is_anomalous'].mean().plot()
# plt.title('Anomaly Distribution Over Time')
# plt.xlabel('Time')
# plt.ylabel('Proportion of Anomalous Traffic')
# plt.show()


Anomaly Statistics:
Total records: 14945
Anomalous records: 10
Percentage of anomalous data: 0.07%

After including additional suspicious behavior:
Anomalous records: 72
Percentage of anomalous data: 0.48%


In [35]:
# Display the anomalous activity from the dataset
anomalous_activity = df_with_anomalies[df_with_anomalies['is_anomalous']]
print("Anomalous Activity:")
print(anomalous_activity)

Anomalous Activity:
              date      time              ip method path  protocol  status  \
1522   07/Jan/2025  00:24:57   82.84.217.223    GET    /  HTTP/1.1     200   
1953   07/Jan/2025  11:11:33    82.139.39.57    GET    /  HTTP/1.1     200   
2012   07/Jan/2025  11:50:12   82.212.93.186    GET    /  HTTP/1.1     200   
2798   08/Jan/2025  05:00:19  82.120.247.227    GET    /  HTTP/1.1     200   
3952   08/Jan/2025  20:03:07  82.155.213.224    GET    /  HTTP/1.1     200   
...            ...       ...             ...    ...  ...       ...     ...   
12414  15/Jan/2025  00:23:27    82.33.221.12    GET    /  HTTP/1.1     200   
12597  15/Jan/2025  07:59:20  185.95.199.120    GET    /  HTTP/1.1     200   
13512  15/Jan/2025  16:20:16    81.43.161.77    GET    /  HTTP/1.1     200   
13811  15/Jan/2025  20:54:20    81.59.150.62    GET    /  HTTP/1.1     200   
14743  16/Jan/2025  14:41:45     81.82.57.31    GET    /  HTTP/1.1     200   

      referrer                             

## Podsumowanie analizy anomalii

Na podstawie powyzszej analizy zidentyfikowano następujące typy ataków i anomalii:

1. **SQL Injection Attacks**:
   - Próby wstrzyknięcia złośliwego kodu SQL
   - Szczegóły dla każdego atakującego IP zostały wyświetlone powyżej
   - Zawierają dokładne przedziały czasowe i przykładowe payloady

2. **Brute Force Attacks**:
   - Próby zgadnięcia hasła poprzez wielokrotne próby logowania
   - Dla każdego atakującego IP pokazano:
     * Liczbę prób
     * Dokładny przedział czasowy
     * Zaatakowane konta użytkowników

3. **Dodatkowe anomalie**:
   - Podejrzane wzorce ruchu (ponad 7 requestów na minutę)
   - Nietypowe User-Agent strings
   - Nienaturalne wzorce dostępu do endpointów

## Rekomendowany podział danych

Bazując na przeprowadzonej analizie, zidentyfikowano około 72 nietypowych zapytań, czyli ok. 0.48% przechwyconego ruchu sieciowego, najwięcej anomalii nastąpiło w dniach 8.01 oraz 14.01. 

### Podział danych ze względu na architekturę modelu AI

Jako detektor anomalii wytrenuję model LSTM-RNN, który dzięki wykorzystaniu ukrytych stanów, sekwencyjnie analizuje dane biorąc pod uwagę poprzednie wejścia. 

Model zostanie nauczony wzorców normalnego ruchu w paradygmacie self-supervised learning. W tym celu najlepsze wyniki uzyskam, poprzez oddzielenie ruchu z anomaliami od typowego ruchu sieciowego. 
 
- 80% - zbiór treningow oraz walidacyjny
   - 80% * 88% = 70,4% : zbiór testowy
   - 80% * 12% = 9,6% : zbiór walidacyjny
- 20% - zbiór testowy zawierający większość ruchu z anomaliami
   
W celu wybrania ruchu z największą ilością anomalii wybrałem 2 dni, które mają największą proporcję podejrzanego ruchu do ruchu normalnego. Wybrane dni to: 08.01.2025 oraz 14.01.2025


# Trenowanie modelu
W folderze RNN znajdują się definicje modelu LSTM-RNN, wytrenowane wagi oraz skrypt do uruchamiania modelu dla nowych danych.

## Proces trenowania
Przed przystąpieniem do trenowania nalezy rozdzielić zbiór danych na treningowy oraz testowy. Jak widać w folderze RNN/weights znajdują się takze wagi modelu trenowanego na całości danych (80% train / 20% val)

### Podział danych
Zgodnie z powyzszą analizą wybrałem dni 8 i 14 stycznia jako testowe, gdyz są najbardziej podejrzane. W tym celu mozna uzyć skryptu `scripts/split_data.py`

In [3]:
!python scripts/split_data.py

Split files already exist in data/splits


### Trenowanie modelu
Plik `RNN/train.py`posiada zarówno definicję modelu, jak i proces trenowania

In [4]:
!python RNN/train.py

Traceback (most recent call last):
  File "/Users/70ziko/projekty/RoarING/solutions/2/RNN/train.py", line 3, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


### Testowanie na nowych danych (lub oddzielonych)
W tym celu mozna uzyć skryptu `RNN/inference.py`, który zapisze wyniki w `RNN/anomaly_detection_results.csv`. Podsumowanie zostanie wyświetlone w stdout:
```bash
Summary:
--------------------------------------------------
Detected 130 anomalies in 2603 logs
Anomaly threshold: 1.9764
```

In [5]:
!python RNN/inference.py

Traceback (most recent call last):
  File "/Users/70ziko/projekty/RoarING/solutions/2/RNN/inference.py", line 1, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


# Analiza predykcji
Statystyki i podgląd predykcji mozna wygenerować uywając skryptu: `scripts/analyze_results.py`

In [8]:
!python scripts/analyze_results.py

=== Anomaly Detection Summary ===
Total logs analyzed: 2,603
Total anomalies detected: 130 (5.0% of logs)
Average anomaly score: 0.6475
Anomaly score threshold: 1.9802

=== Temporal Distribution ===
Peak anomaly hour: 12:00 (16 anomalies)

=== IP Address Analysis ===
Top 5 IPs with most anomalies:
  81.221.120.178: 2 anomalies
  81.246.224.238: 2 anomalies
  81.214.169.226: 2 anomalies
  82.143.172.44: 2 anomalies
  82.141.153.154: 2 anomalies

=== Endpoint Analysis ===
Most attacked endpoints:
  /wylogowanie: 79 anomalies
  /transakcja: 29 anomalies
  /logowanie: 20 anomalies
  /: 1 anomalies
  /rejestracja: 1 anomalies

=== Sample Anomalies ===
Top 3 anomalies by score:
--------------------------------------------------------------------------------
Time: 08/Jan/2025 15:02:02
IP: 81.246.224.238
Request: POST /logowanie
Status: 400
Payload: login=adaro6794&password=&action=login
Anomaly Score: 5.1001
--------------------------------------------------------------------------------
Time